In [1]:
"""
    This notebook captures the process by which de-identification is ran against the AoUS bigquery database
    @TODO:
    - Need to add some form of logs to know what is going on at all times
    - 
"""
from __future__ import division
import json
from google.cloud import bigquery as bq

from deid import *
client = bq.Client.from_service_account_json('/home/steve/git/rdc/deid/config/account/account.json')

In [ ]:
#
# This section performs suppression of PPI on a meta-table
# @TODO INsure the way in which the PPI are identified is correct. You should make sure the dates aren't suppressed.
# Dates will have to be shifted ()
#
handler = SuppressPPI(client=client)
r = handler.can_do('raw')
handler.do(i_dataset='raw',table_name='observation', o_dataset='out')

In [ ]:
#
# This section performs a suppression of fields from a non-meta table
# A configureation should be provided to the class
#
config = {"raw":{"person":['year_of_birth','month_of_birth','day_of_birth']}}
h = SuppressEHR(client=client,config = config)
h.can_do('raw','person')
h.do(i_dataset='raw',o_dataset='out',table_name='person')

In [2]:
#
# This section will perform a date shifting on PPI and EHR
# The date shifting operation will (to a certain degree) include suppression
#
h = Shift(client=client,vocabulary_id='PPI',concept_class_id=['Question','PPI Modifier'])
[h.vocabulary_id,h.concept_sql]
r = h.can_do('raw')
h.do(i_dataset='raw',table_name='observation',o_dataset='out')
# h.sql

 *** done 


            SELECT x.observation_id,x.person_id,x.observation_concept_id,x.observation_date,x.observation_datetime,x.observation_type_concept_id,x.value_as_number,x.value_as_concept_id,x.qualifier_concept_id,x.unit_concept_id,x.provider_id,x.visit_occurrence_id,x.observation_source_concept_id,x.unit_source_value,x.qualifier_source_value,x.value_source_concept_id,x.value_source_value,x.questionnaire_response_id, x.observation_source_value, DATE_DIFF(cast(x.value_as_string as date), cast(y.value_as_string as date), DAY) as value_as_string
            FROM raw.observation   x INNER JOIN (SELECT value_as_string,person_id from raw.observation where observation_source_value = 'ExtraConsent_TodaysDate') y ON y.person_id = x.person_id

            WHERE x.person_id = 562270 AND
                x.observation_source_value in (
                select concept_code from raw.concept
                where
                vocabulary_id = 'PPI' AND REGEXP_CONTAINS(concept_code,'(date|Date|

In [ ]:
ref = client.dataset('raw').table('observation')
client.get_table(ref).schema